##Purpose

The goal of this file is to quickly read in raw data from a file, give the user an overview of the contents within, and then convert each column to have the correct datatype (i.e from "$1,234.55" to 1234.55) with minimal manual involvement. This is most suitable for smaller size datasets (< 1million rows).

In [1]:
import pandas as pd
import numpy as np
import re
import os
from multiprocessing import Pool
from datetime import datetime
from sklearn.model_selection import train_test_split

##Reading In/Summarizing Data

###Helper Functions:

1. CatFiles(): Reads in/concatenates atomized data files into a single dataframe
2. NULLChecker(): prompts the user on which columns/rows contain null data
4. TextLook(): Finds unique values for each column by default, or columns you specify in a dataframe
3. InitialDisplay(): shows summary information about the dataframe like datatypes,summary stats, null values, unique values

In [2]:
#Read in/concatenate Files
def CatFiles(wd,files):
    df=pd.DataFrame()
    for file in files: df=pd.concat([df,pd.read_csv(wd+file,sep=',')])
    return df

In [3]:
#returns Null rows
def NullRows(df,res=False):
    df=df[df.isnull().any(axis=1)]
    if len(df)>0:res=df
    return(res)

#check for null values, returns columns where they are or False
def NULLChecker(df,res=False):
    df=df.replace(r'^\s*$', np.nan, regex=True)
    res1=df.isnull().values.any()
    if res1 == True:
        res=df.columns[df.isna().any()].tolist()
        print("These columns have null values:\n",res)   
        val=NullRows(data)
        print("These are",len(val)," null rows:\n",val)

In [9]:
#Find unique values for each column/columns you specify in a dataframe
def TextLook(df,columns=False):
    if columns == False: columns=df.columns
    for column in columns:
        n_elts = len(set(df[column]))
        print(column,":",n_elts,"unique values")

In [21]:
#shows summary info
def InitialDisplay(df,columns=False):
    print("Head:\n",df.head())
    print("\nInfo:")
    print(df.info())
    print("\nDescription:\n",df.describe(),"\n")
    NULLChecker(df)#display null information
    print("\nUnique Values:")
    print(TextLook(df,columns))

##Cleaning Up Dataset

###Helper Functions:

1. Replacer(): Removes special characters from string in numpy array, strips white space
2. DataTypeConverter(): Converts strings to date, float, and int datatypes.
3. DataCleaner(): Converts dataframe to correct datatypes, strips white space/special characaters. Returns converted dataframe.
4. TrainValidateSplit(): Splits dataframe into training/test/final validation datasets.

In [11]:
#replace string in numpy array, strip white space
def Replacer(string):
    try:
        return string.replace('$','').replace('&','').replace('#','').strip()
    except:
        return string

In [12]:
#Converts strings to date, float, int types
def DataTypeConverter(df,date_format):
    try:
        return list(map(float, df)) #try first if float
    except:
        try:
            return list(map(int, df)) #try then if int
        except:
            try: #then try if date
                return list(map(lambda x: pd.to_datetime(x,format=date_format),df))
            except:
                return df

In [28]:
#Converts dataframe to correct datatypes, strips white space/special characaters. Returns converted dataframe
def DataCleaner(df,date_format='%m/%d/%Y'):
    #flag non numeric columns
    non_numbers_columns=df.select_dtypes(exclude=[np.number]).columns.values
    numeric_columns=df.columns[df.columns.isin(non_numbers_columns)==False].values
    print("These columns have non-numeric data:\n",non_numbers_columns,"\n")
    #remove special characters for non-numeric columns
    non_numbers=np.array(list(map(lambda y: list(map(lambda x: Replacer(x),y)),np.array(df[non_numbers_columns]))))
    #converting to appropriate datatypes
    new_df=np.array(list(map(lambda column: DataTypeConverter(non_numbers[:,column],date_format),range(0,non_numbers.shape[1])))).T
    print("This is the new list of converted values:\n",new_df,"\n")
    #combining numeric and non-numeric/converted columns
    new_df = pd.concat([pd.DataFrame(new_df), df[numeric_columns]], axis=1)
    new_df.columns = np.append(non_numbers_columns,numeric_columns)
    print("This is the new converted dataframe:\n",new_df,"\n")
    print('\n',new_df.info())
    return(new_df)

In [14]:
#Split training/test/final_test dataset
def TrainValidateSplit(df,target_value,shuffle=False,ratio=0.2):
    X=df.drop(target_value,axis=1)
    y=df[target_value]
    x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=ratio,shuffle=shuffle)
    x_test,x_test_final,y_test,y_test_final=train_test_split(x_test,y_test,test_size=0.5,shuffle=shuffle)
    return x_train,x_test,x_test_final,y_train,y_test,y_test_final

##Example Dataset

1. Read in data
2. Display Summary Information
3. Clean-Up/Convert to appropriate datatypes
4. Partition training/testing/final validation set

In [35]:
#read in data files
wd=os.getcwd()
companies=['aapl.us.txt','msft.us.txt','amzn.us.txt','goog.us.txt']
data=CatFiles(wd+"/stocks/Stocks/",companies)

In [22]:
#getting summary Information
InitialDisplay(data)

Head:
    quarter stock       date    open    high     low   close     volume  \
0        1    AA   1/7/2011  $15.82  $16.72  $15.78  $16.42  239655616   
1        1    AA  1/14/2011  $16.71  $16.71  $15.64  $15.97  242963398   
2        1    AA  1/21/2011  $16.19  $16.38  $15.60  $15.79  138428495   
3        1    AA  1/28/2011  $15.87  $16.63  $15.82  $16.13  151379173   
4        1    AA   2/4/2011  $16.18  $17.39  $16.18  $17.14  154387761   

   percent_change_price  percent_change_volume_over_last_wk  \
0               3.79267                                 NaN   
1              -4.42849                            1.380223   
2              -2.47066                          -43.024959   
3               1.63831                            9.355500   
4               5.93325                            1.987452   

   previous_weeks_volume next_weeks_open next_weeks_close  \
0                    NaN          $16.71           $15.97   
1            239655616.0          $16.19       

In [29]:
#Data Preperation
new_df=DataCleaner(data,date_format='%m/%d/%Y')

These columns have non-numeric data:
 ['stock' 'date' 'open' 'high' 'low' 'close' 'next_weeks_open'
 'next_weeks_close'] 

This is the new list of converted values:
 [['AA' Timestamp('2011-01-07 00:00:00') 15.82 ... 16.42 16.71 15.97]
 ['AA' Timestamp('2011-01-14 00:00:00') 16.71 ... 15.97 16.19 15.79]
 ['AA' Timestamp('2011-01-21 00:00:00') 16.19 ... 15.79 15.87 16.13]
 ...
 ['XOM' Timestamp('2011-06-10 00:00:00') 80.93 ... 79.78 80.0 79.02]
 ['XOM' Timestamp('2011-06-17 00:00:00') 80.0 ... 79.02 78.65 76.78]
 ['XOM' Timestamp('2011-06-24 00:00:00') 78.65 ... 76.78 76.88 82.01]] 

This is the new converted dataframe:
     stock       date   open   high    low  close next_weeks_open  \
0      AA 2011-01-07  15.82  16.72  15.78  16.42           16.71   
1      AA 2011-01-14  16.71  16.71  15.64  15.97           16.19   
2      AA 2011-01-21  16.19  16.38   15.6  15.79           15.87   
3      AA 2011-01-28  15.87  16.63  15.82  16.13           16.18   
4      AA 2011-02-04  16.18  17.3

In [30]:
#sorting by date
new_df=new_df.sort_values(['date'],ascending=True)

In [37]:
#partioning train/test/validate set
x_train,x_test,x_test_final,y_train,y_test,y_test_final=train_validate_split(new_df,
                                                        target_value="close",
                                                        shuffle=False,
                                                        ratio=0.2)